In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, regularizers

### Loading the Data

In [2]:
cvd_data = pd.read_csv(r"Datasets\Analyzed\Analyzed_CVD_Prediction.csv")
cvd_data.head()

,heart_rate,oxygen_saturation,respiratory_rate,temperature,Label
0,93.0,98.0,39.0,36.0,1
1,97.0,98.0,16.0,36.5,0
2,82.0,97.0,25.0,36.3,1
3,98.0,96.0,16.0,36.5,1
4,98.0,99.0,28.0,37.2,1


# Training a DL Model using Keras with 2 Layers

In [3]:
# Split the data into training and validation sets
train_data = cvd_data.sample(frac=0.8, random_state=0)
val_data = cvd_data.drop(train_data.index)

# Preprocess the data
train_labels = train_data.pop('Label')
val_labels = val_data.pop('Label')

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)

# Define the model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=50,
                    validation_data=(val_data, val_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(val_data, val_labels)

print('Test accuracy:', test_acc)


Epoch 1/50
346/346 [==============================] - 6s 5ms/step - loss: 0.2841 - accuracy: 0.8521 - val_loss: 0.2059 - val_accuracy: 0.8618
Epoch 2/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1954 - accuracy: 0.8819 - val_loss: 0.1817 - val_accuracy: 0.8875
Epoch 3/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1758 - accuracy: 0.9024 - val_loss: 0.1665 - val_accuracy: 0.9121
Epoch 4/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1630 - accuracy: 0.9174 - val_loss: 0.1605 - val_accuracy: 0.9269
Epoch 5/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1565 - accuracy: 0.9219 - val_loss: 0.1572 - val_accuracy: 0.9092
Epoch 6/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1510 - accuracy: 0.9257 - val_loss: 0.1520 - val_accuracy: 0.9219
Epoch 7/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1477 - accuracy: 0.9267 - val_loss: 0.1435 - val_accuracy: 0.9331
Epoch 

#### These scores seems good to be deploy, so it will be converted into a TFLite model

### Converting this model to TFLite Model

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]  # <-- ensure Pi compatibility
tflite_model = converter.convert()

with open('Models/CVD_Classification_Model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\SANDIN~1\AppData\Local\Temp\tmpuh7keft9\assets
